# Final

In [1]:
# data tools
import os
import re
import json 
import string
import warnings 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
warnings.filterwarnings('ignore')
from subprocess import check_output
import contractions

# nltk and sklearn
import nltk
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# tensorflow and keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


# Get the dataset (Amazon Video Game Reviews)

In [3]:
#!curl https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz -o games.tsv.gz
#!gzip -d games.tsv.gz

# Load into Pandas DataFrame 

There are a small number of lines in this file that don't parse properly

In [4]:
games = pd.read_csv('games.tsv', delimiter = '\t', error_bad_lines=False)

b'Skipping line 20630: expected 15 fields, saw 22\nSkipping line 28172: expected 15 fields, saw 22\nSkipping line 54791: expected 15 fields, saw 22\n'
b'Skipping line 75419: expected 15 fields, saw 22\nSkipping line 104832: expected 15 fields, saw 22\n'
b'Skipping line 138464: expected 15 fields, saw 22\nSkipping line 194849: expected 15 fields, saw 22\n'
b'Skipping line 201568: expected 15 fields, saw 22\nSkipping line 242567: expected 15 fields, saw 22\n'
b'Skipping line 493585: expected 15 fields, saw 22\nSkipping line 502478: expected 15 fields, saw 22\n'
b'Skipping line 660750: expected 15 fields, saw 22\n'


# Examine Data

In [5]:
games.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31


In [6]:
games.dtypes

marketplace          object
customer_id           int64
review_id            object
product_id           object
product_parent        int64
product_title        object
product_category     object
star_rating           int64
helpful_votes         int64
total_votes           int64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtype: object

The data types all seem to be correct

In [7]:
games.describe()

,customer_id,product_parent,star_rating,helpful_votes,total_votes
count,1.780268e+06,1.780268e+06,1.780268e+06,1.780268e+06,1.780268e+06
mean,2.803021e+07,4.954251e+08,4.059375e+00,2.263971e+00,3.764128e+00
std,1.562851e+07,2.909145e+08,1.355964e+00,1.919211e+01,2.247177e+01
min,1.001800e+04,3.140000e+03,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.429715e+07,2.452708e+08,4.000000e+00,0.000000e+00,0.000000e+00
50%,2.732141e+07,4.920909e+08,5.000000e+00,0.000000e+00,1.000000e+00
75%,4.274834e+07,7.485890e+08,5.000000e+00,1.000000e+00,3.000000e+00
max,5.309656e+07,9.999685e+08,5.000000e+00,1.049800e+04,1.078000e+04


In [8]:
games.star_rating.value_counts()

5    1023167
4     317460
1     191633
3     153451
2      94557
Name: star_rating, dtype: int64

Star ratings tend to be fairly high on average with a mean of 4, most reviews have no helpful_votes but some reviews have a lot of helpful votes

In [9]:
games.isna().sum()

marketplace           0
customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
product_category      0
star_rating           0
helpful_votes         0
total_votes           0
vine                  0
verified_purchase     0
review_headline      28
review_body          59
review_date          27
dtype: int64

There are a very small number of NA values, they make up less than .01% of the data so dropping those rows should not have a substantial impact

In [10]:
games.dropna(inplace=True)

In [11]:
games.isna().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
dtype: int64

In order to get everything the customer wrote for the review, we are combining the 'review_headline' and 'review_body'. There are, however, a large number of reviews where the review headline is just a restatement of the star rating, this is information that is already captured elsewhere so those headlines are being ignored. 

In [12]:
games['review_full'] = np.where(games['review_headline'].str.lower().str.contains('star'), games['review_body'], 
                                games['review_headline'] + ' '+ games['review_body'])

In [13]:
games.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_full
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31,an amazing joystick. I especially love that yo...
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31,Definitely a silent mouse... Not a single clic...
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31,poor quality work and not as it is advertised.
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31,"good, but could be bettee nice, but tend to sl..."
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31,"Great but flawed. Great amiibo, great for coll..."


We are interested in analyzing whether the review was positive or negative, so a new column is being created which classifies a review as positive if it is 4 stars or greater, negative if it is 2 stars or less, and neutral otherwise

In [14]:
games['Sentiment_target'] = np.where(games.star_rating > 3, 'Positive', np.where(games.star_rating < 3, 'Negative', 'Neutral'))

In [15]:
games.Sentiment_target.value_counts()

Positive    1340545
Negative     286166
Neutral      153443
Name: Sentiment_target, dtype: int64

# Preprocessing and visualization

Define a pre-processing function to expand contractions, lemmatize the text and remove stopwords (this takes some time to run)

In [16]:
def preprocess(text):
    
    text = ' '.join([contractions.fix(word) for word in text.split(' ')])
    
    words = nltk.tokenize.word_tokenize(text)
        
    stopwords = nltk.corpus.stopwords.words('english')
    words = [word for word in words if word not in stopwords+['br','wa']+list(string.punctuation)]
    
    lem = nltk.stem.WordNetLemmatizer()
    words = [lem.lemmatize(word) for word in words]

   
    return ' '.join(words)

In [17]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jor34\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jor34\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jor34\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
#games['review_clean'] = games.review_full.apply(preprocess)
#games.sample(10)

# read from pickle rather than run preprocess to save time
games = pd.read_pickle('games.pkl')

In [ ]:
games

In [ ]:
#games.to_pickle('games.pkl')

In [ ]:
from wordcloud import WordCloud

In [ ]:
full_text = ' '.join(games.review_clean)
wordcloud = WordCloud().generate(full_text)

plt.rcParams["figure.figsize"] = (20,15)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

This is a word cloud showing word frequency across all reviews. The most frequent words here definitely trend towards the positive side with 'great' showing up frequently, this could be because over 50% of all of the reviews are 5 star reviews, and the vast majority of reviews are positive.

In [ ]:
full_text = ' '.join(games.review_clean[games.Sentiment_target=='Positive'])
wordcloud = WordCloud().generate(full_text)

plt.rcParams["figure.figsize"] = (20,15)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

This is a word cloud showing word frequency is positive reviews.It looks very similar tothe word cloud showing all reviews, but words expressing positive sentiment like 'good', 'great', and 'love' are more common. 

In [ ]:
full_text = ' '.join(games.review_clean[games.Sentiment_target=='Negative'])
wordcloud = WordCloud().generate(full_text)

plt.rcParams["figure.figsize"] = (20,15)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Modeling

In [ ]:
#This seems unneccesary now
#games = pd.read_csv('games.tsv', sep='\t', error_bad_lines=False)
#games = games[['review_body','star_rating']]
#row_select_1 = games['star_rating'] < 3
#row_select_2 = games['star_rating'] == 3
#games['sentiment'] = pd.Series(['Positive']*len(games.index))
#games.loc[row_select_1,'sentiment'] = 'Negative'
#games.loc[row_select_2,'sentiment'] = 'Neutral'
#games = games[['review_body','sentiment']]
#games.columns = ['text', 'sentiment']
#games

In [ ]:
# Sentiment_target review_clean
games = games[['review_clean','Sentiment_target']]
games.columns = ['text', 'sentiment']
games

In [ ]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 2

In [ ]:
games = games[games.sentiment != "Neutral"]
games['text'] = games['text'].apply(lambda x: str(x).lower())
games['text'] = games['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(games[ games['sentiment'] == 'Positive'].size)
print(games[ games['sentiment'] == 'Negative'].size)

for idx,row in games.iterrows():
    row[0] = row[0].replace('rt',' ')
    
tokenizer = Tokenizer(nb_words = max_features, split=' ')
tokenizer.fit_on_texts(games['text'].values)
X = tokenizer.texts_to_sequences(games['text'].values)
X = pad_sequences(X, maxlen = maxlen)

In [ ]:
Y = pd.get_dummies(games['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,shuffle=True, validation_data=(X_test, Y_test))

In [ ]:
yhat = model.predict(X_test, verbose = 2, batch_size = batch_size)
from sklearn import metrics
print(metrics.classification_report(Y_test[:,1], np.round(yhat[:,1]) ,target_names = ["negative", "positive"]))

In [ ]:
# score = ['negative', 'positive']

# def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Greys):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(set(score)))
#     plt.xticks(tick_marks, score, rotation=45)
#     plt.yticks(tick_marks, score)
#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
    
# # Compute confusion matrix
# cm = confusion_matrix(Y_test[:,1], np.round(yhat[:,1]))
# np.set_printoptions(precision=2)

# plt.figure()
# plot_confusion_matrix(cm)    

# cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# plt.figure()
# plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

# plt.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_conv_lstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_conv_lstm.h5")

In [ ]:
score = ['negative', 'positive']

# Compute confusion matrix
cm = confusion_matrix(Y_test[:,1], np.round(yhat[:,1]))
np.set_printoptions(precision=2)

In [ ]:
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
plt.subplots(figsize=(15,8))
sns.set(font_scale=1.2)
conf_plot = sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

In [ ]:
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.subplots(figsize=(15,8))
sns.set(font_scale=1.2)
conf_plot_norm = sns.heatmap(cm_normalized, annot=labels, fmt='', cmap='Blues')